In [1]:
# %tensorflow_version 2.x
import tensorflow as tf
import string
import requests

In [2]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [3]:
# response.text

In [4]:
data = response.text.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [5]:
data[253]

'  From fairest creatures we desire increase,'

In [6]:
data = data[253:]

In [7]:
data[0]

'  From fairest creatures we desire increase,'

In [8]:
len(data)

124204

In [9]:
data = " ".join(data)

In [10]:
# data

In [11]:
def clean_text(doc):
    tokens = doc.split()
    table = str.maketrans('','',string.punctuation)
    tokens = [w.translate(table) for w in tokens ]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return tokens

In [12]:
tokens = clean_text(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [13]:
print(len(tokens))

898199


In [14]:
len(set(tokens))

27956

In [15]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > 200000:
        break
    
print(len(lines))

199951


In [16]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [17]:
print(tokens[0])
print(tokens[1])
print(tokens[50])
# print(tokens[0:])

from
fairest
self


In [18]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

In [19]:
len(lines[0])

302

In [20]:
# len(clean_text(lines[8])) # ONLY FOR TESTING

BUILD LSTM MODEL AND PREPARE X AND Y

first 50 words will be used to predict the 51st word

In [21]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [23]:
sequences

[[47,
  1408,
  1264,
  37,
  451,
  1406,
  9,
  2766,
  1158,
  1213,
  171,
  132,
  269,
  20,
  24,
  1,
  4782,
  87,
  30,
  98,
  4781,
  18,
  715,
  1263,
  171,
  211,
  18,
  829,
  20,
  27,
  3807,
  4,
  214,
  121,
  1212,
  153,
  13004,
  31,
  2765,
  1847,
  16,
  13003,
  13002,
  754,
  7,
  3806,
  99,
  2430,
  466,
  31,
  307],
 [1408,
  1264,
  37,
  451,
  1406,
  9,
  2766,
  1158,
  1213,
  171,
  132,
  269,
  20,
  24,
  1,
  4782,
  87,
  30,
  98,
  4781,
  18,
  715,
  1263,
  171,
  211,
  18,
  829,
  20,
  27,
  3807,
  4,
  214,
  121,
  1212,
  153,
  13004,
  31,
  2765,
  1847,
  16,
  13003,
  13002,
  754,
  7,
  3806,
  99,
  2430,
  466,
  31,
  307,
  31],
 [1264,
  37,
  451,
  1406,
  9,
  2766,
  1158,
  1213,
  171,
  132,
  269,
  20,
  24,
  1,
  4782,
  87,
  30,
  98,
  4781,
  18,
  715,
  1263,
  171,
  211,
  18,
  829,
  20,
  27,
  3807,
  4,
  214,
  121,
  1212,
  153,
  13004,
  31,
  2765,
  1847,
  16,
  13003,
  13002,
 

In [24]:
sequences = np.array(sequences)
sequences

array([[  47, 1408, 1264, ...,  466,   31,  307],
       [1408, 1264,   37, ...,   31,  307,   31],
       [1264,   37,  451, ...,  307,   31, 1582],
       ...,
       [  33,   80, 5197, ...,  215,   44,   30],
       [  80, 5197,  103, ...,   44,   30, 1332],
       [5197,  103, 1846, ...,   30, 1332,    2]])

In [25]:
X, y = sequences[:, :-1], sequences[:,-1]

In [26]:
X[0]

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31])

In [27]:
y[0]

307

In [28]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'to': 4,
 'of': 5,
 'you': 6,
 'a': 7,
 'my': 8,
 'that': 9,
 'in': 10,
 'is': 11,
 'not': 12,
 'it': 13,
 'for': 14,
 'me': 15,
 'with': 16,
 'he': 17,
 'his': 18,
 'your': 19,
 'but': 20,
 'be': 21,
 'this': 22,
 'have': 23,
 'as': 24,
 'him': 25,
 'so': 26,
 'thou': 27,
 'will': 28,
 'what': 29,
 'by': 30,
 'thy': 31,
 'no': 32,
 'are': 33,
 'all': 34,
 'her': 35,
 'do': 36,
 'we': 37,
 'if': 38,
 'our': 39,
 'or': 40,
 'shall': 41,
 'thee': 42,
 'which': 43,
 'on': 44,
 'lord': 45,
 'o': 46,
 'from': 47,
 'good': 48,
 'more': 49,
 'sir': 50,
 'was': 51,
 'they': 52,
 'well': 53,
 'at': 54,
 'would': 55,
 'when': 56,
 'now': 57,
 'come': 58,
 'love': 59,
 'th': 60,
 'than': 61,
 'am': 62,
 'then': 63,
 'she': 64,
 'their': 65,
 'them': 66,
 'how': 67,
 'enter': 68,
 'let': 69,
 'did': 70,
 'ill': 71,
 'hath': 72,
 'one': 73,
 'us': 74,
 'know': 75,
 'first': 76,
 'make': 77,
 'had': 78,
 'like': 79,
 'here': 80,
 'upon': 81,
 'there': 82,
 'man': 83,


In [29]:
vocab_size = len(tokenizer.word_index)+1

In [30]:
y = to_categorical(y, num_classes = vocab_size)

In [31]:
X.shape[1]

50

In [32]:
seq_length = X.shape[1]

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100,return_sequences = True))
model.add(LSTM(100))
# model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='softmax'))
model.add(Dense(vocab_size,activation='softmax'))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            650450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
# model.fit(X,y,batch_size=256,epochs=100)

In [37]:
import pickle

In [38]:
with open('predict_model','wb') as f:
    pickle.dump(model,f)

TypeError: cannot pickle 'weakref' object

In [ ]:
# import joblib

In [ ]:
# joblib.dump(model,'model_autotextgeneration')

In [ ]:
import os.path
if os.path.isfile('model_auto_text_generation.h5') is False:
    model.save('model_auto_text_generation.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# new_model = load_model('model_auto_text_generation.h5')
model = load_model('model_auto_text_generation.h5')

In [ ]:
new_model.summary()

In [ ]:
model.summary()

In [ ]:
model.get_weights()

In [ ]:
model.optimizer

In [ ]:
json_string = model.to_json()

In [ ]:
json_string

In [ ]:
from tensorflow.keras.models import model_from_json
model_architecture = model_from_json(json_string)

In [ ]:
model_architecture.summary()

In [ ]:
lines[12343]

In [ ]:
seed_text = lines[12343]

In [ ]:
def generate_text_seq(model,tokenizer,text_seq_length,seed_text,n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded],maxlen = text_seq_length, truncating = 'pre')
        
        y_predic = model.predict_classes(encoded)
        
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predic:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [ ]:
generate_text_seq(model,tokenizer,seq_length,seed_text,10)

In [ ]:
src_txt = "He likes eating samosa. Samosa is a Indian cuisine. Raj likes Samosa."

In [ ]:
generate_text_seq(model,tokenizer,seq_length,src_txt,10)